# VTL Simple analytic function



In [1]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, IntegerType, LongType, DecimalType
import os
from pyspark.sql.functions import lit, count,sum,avg,collect_list,min,max,percentile_approx,stddev_pop,stddev_samp,var_pop,var_samp
from pyspark.sql.window import Window

In [2]:
local = True

if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("VTLAnalytic")\
        .getOrCreate()
else:
    spark = SparkSession.builder\
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("VTLAnalytic")\
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:py3.9.7-spark3.2.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "8g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-06-21 14:40:49,058 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
data=[("A", "XX", 2000, 3, 1.0),
    ("A", "XX", 2001, 4, 9.0),
    ("A", "XX", 2002, 7, 5.0),
    ("A", "XX", 2003, 6, 8.0),
    ("A", "YY", 2000, 9, 3.0),
    ("A", "YY", 2001, 5, 4.0),
    ("A", "YY", 2002, 10, 2.0),
    ("A", "YY", 2003, 5, 7.0)]

schema=StructType([StructField("Id_1",StringType(),True),
                   StructField("Id_2",StringType(),True),
                   StructField("Year",IntegerType(),True),
                   StructField("Me_1",IntegerType(),True),
                   StructField("Me_2",DoubleType(),True)])

df=spark.createDataFrame(data, schema)
df.show()

+----+----+----+----+----+
|Id_1|Id_2|Year|Me_1|Me_2|
+----+----+----+----+----+
|   A|  XX|2000|   3| 1.0|
|   A|  XX|2001|   4| 9.0|
|   A|  XX|2002|   7| 5.0|
|   A|  XX|2003|   6| 8.0|
|   A|  YY|2000|   9| 3.0|
|   A|  YY|2001|   5| 4.0|
|   A|  YY|2002|  10| 2.0|
|   A|  YY|2003|   5| 7.0|
+----+----+----+----+----+



## 1.1 Count

### 1.1.1 Exp1 : window only has partition clause
The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 ) )
```


In [9]:
partition_col_name="Id_1"

 
win_name=Window.partitionBy(partition_col_name)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"count_{target_col_name1}"
new_col_name2=f"count_{target_col_name2}"
df_count=df.withColumn(new_col_name1,count(target_col_name1).over(win_name))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name))
df_count.show()

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         8|         8|
|   A|  XX|2001|   4| 9.0|         8|         8|
|   A|  XX|2002|   7| 5.0|         8|         8|
|   A|  XX|2003|   6| 8.0|         8|         8|
|   A|  YY|2000|   9| 3.0|         8|         8|
|   A|  YY|2001|   5| 4.0|         8|         8|
|   A|  YY|2002|  10| 2.0|         8|         8|
|   A|  YY|2003|   5| 7.0|         8|         8|
+----+----+----+----+----+----------+----------+



### 1.1.2 Exp 2: window has partition and order by

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Id_2) )
```


In [12]:

win_name_with_order=Window.partitionBy(partition_col_name).orderBy("Id_2")

df_count_with_order=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order))

df_count_with_order.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         4|         4|
|   A|  XX|2001|   4| 9.0|         4|         4|
|   A|  XX|2002|   7| 5.0|         4|         4|
|   A|  XX|2003|   6| 8.0|         4|         4|
|   A|  YY|2000|   9| 3.0|         8|         8|
|   A|  YY|2001|   5| 4.0|         8|         8|
|   A|  YY|2002|  10| 2.0|         8|         8|
|   A|  YY|2003|   5| 7.0|         8|         8|
+----+----+----+----+----+----------+----------+



### 1.1.3 Exp 3: window has partition, order by and data point

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Id_2 data points between -2 and 2) )
```

In [14]:
win_name_with_order_dp=Window.partitionBy(partition_col_name).orderBy("Id_2").rowsBetween(-2,2)

df_count_with_order_dp=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order_dp))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order_dp))

df_count_with_order_dp.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         3|         3|
|   A|  XX|2001|   4| 9.0|         4|         4|
|   A|  XX|2002|   7| 5.0|         5|         5|
|   A|  XX|2003|   6| 8.0|         5|         5|
|   A|  YY|2000|   9| 3.0|         5|         5|
|   A|  YY|2001|   5| 4.0|         5|         5|
|   A|  YY|2002|  10| 2.0|         4|         4|
|   A|  YY|2003|   5| 7.0|         3|         3|
+----+----+----+----+----+----------+----------+



In [ ]:
### 1.1.4 Exp 4: window has partition, order by and range 

The vtl query:

```text
res := count ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [17]:
win_name_with_order_range=Window.partitionBy(partition_col_name).orderBy("Year").rangeBetween(-1,1)

df_count_with_order_range=df.withColumn(new_col_name1,count(target_col_name1).over(win_name_with_order_range))\
           .withColumn(new_col_name2,count(target_col_name2).over(win_name_with_order_range))

df_count_with_order_range.show(20)

+----+----+----+----+----+----------+----------+
|Id_1|Id_2|Year|Me_1|Me_2|count_Me_1|count_Me_2|
+----+----+----+----+----+----------+----------+
|   A|  XX|2000|   3| 1.0|         4|         4|
|   A|  YY|2000|   9| 3.0|         4|         4|
|   A|  XX|2001|   4| 9.0|         6|         6|
|   A|  YY|2001|   5| 4.0|         6|         6|
|   A|  XX|2002|   7| 5.0|         6|         6|
|   A|  YY|2002|  10| 2.0|         6|         6|
|   A|  XX|2003|   6| 8.0|         4|         4|
|   A|  YY|2003|   5| 7.0|         4|         4|
+----+----+----+----+----+----------+----------+



In [6]:
df.printSchema()

root
 |-- Id_1: string (nullable = true)
 |-- Id_2: string (nullable = true)
 |-- Id_3: integer (nullable = true)
 |-- Me_1: integer (nullable = true)
 |-- Me_2: integer (nullable = true)



## 1.2 Sum

### 1.2.1 Exp1 : window only has partition clause
The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 ) )
```


In [19]:
partition_col_name="Id_1"
win_name=Window.partitionBy(partition_col_name)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      49|    39.0|
|   A|  XX|2001|   4| 9.0|      49|    39.0|
|   A|  XX|2002|   7| 5.0|      49|    39.0|
|   A|  XX|2003|   6| 8.0|      49|    39.0|
|   A|  YY|2000|   9| 3.0|      49|    39.0|
|   A|  YY|2001|   5| 4.0|      49|    39.0|
|   A|  YY|2002|  10| 2.0|      49|    39.0|
|   A|  YY|2003|   5| 7.0|      49|    39.0|
+----+----+----+----+----+--------+--------+



### 1.2.2 Exp 2: window has partition and order by

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Id_2) )
```

In [20]:
partition_col_name="Id_1"
win_name=Window.partitionBy(partition_col_name).orderBy("Id_2")

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      20|    23.0|
|   A|  XX|2001|   4| 9.0|      20|    23.0|
|   A|  XX|2002|   7| 5.0|      20|    23.0|
|   A|  XX|2003|   6| 8.0|      20|    23.0|
|   A|  YY|2000|   9| 3.0|      49|    39.0|
|   A|  YY|2001|   5| 4.0|      49|    39.0|
|   A|  YY|2002|  10| 2.0|      49|    39.0|
|   A|  YY|2003|   5| 7.0|      49|    39.0|
+----+----+----+----+----+--------+--------+



### 1.2.3 Window with orderby without partition, 

The vtl query:

```text
DS_r := sum ( ds1 over ( order by Id_1, Id_2, Year ) )
``` 

As there is no partition, so the whole data frame is considered as one partition. So the sum become the rolling sum of the whole data set.
This is not recommended, because in spark, one partition means only one worker will do the job, the rest of the worker in the cluster will be idle. It's no longer distributed calculation.

In [23]:
partition_col_names=["Id_1","Id_2","Year"]
win_name=Window.orderBy(partition_col_names)

target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|       3|     1.0|
|   A|  XX|2001|   4| 9.0|       7|    10.0|
|   A|  XX|2002|   7| 5.0|      14|    15.0|
|   A|  XX|2003|   6| 8.0|      20|    23.0|
|   A|  YY|2000|   9| 3.0|      29|    26.0|
|   A|  YY|2001|   5| 4.0|      34|    30.0|
|   A|  YY|2002|  10| 2.0|      44|    32.0|
|   A|  YY|2003|   5| 7.0|      49|    39.0|
+----+----+----+----+----+--------+--------+



2022-06-29 12:48:17,965 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-29 12:48:17,965 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-06-29 12:48:18,069 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


### 1.2.4 Exp : window has partition, order by and data point

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Id_2 data points between -2 and 2) )
```

In [27]:
partition_col_name=["Id_1"]
order_col_name=["Id_2"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_col_name).rowsBetween(-2,2)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name)) \
         .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
df_sum.show()


+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      14|    15.0|
|   A|  XX|2001|   4| 9.0|      20|    23.0|
|   A|  XX|2002|   7| 5.0|      29|    26.0|
|   A|  XX|2003|   6| 8.0|      31|    29.0|
|   A|  YY|2000|   9| 3.0|      37|    22.0|
|   A|  YY|2001|   5| 4.0|      35|    24.0|
|   A|  YY|2002|  10| 2.0|      29|    16.0|
|   A|  YY|2003|   5| 7.0|      20|    13.0|
+----+----+----+----+----+--------+--------+



### 1.2.5 Exp : window has partition, order by and range 

The vtl query:

```text
res := sum ( ds1 over ( partition by Id_1 order by Year range between -1 and 1) )
```

In [30]:
partition_col_name=["Id_1"]
order_col_name=["Year"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_col_name).rangeBetween(-1,1)
target_col_name1="Me_1"
target_col_name2="Me_2"
new_col_name1=f"sum_{target_col_name1}"
new_col_name2=f"sum_{target_col_name2}"

df_sum=df.withColumn(new_col_name1,sum(target_col_name1).over(win_name))\
        .withColumn(new_col_name2,sum(target_col_name2).over(win_name))
                     
df_sum.show()

+----+----+----+----+----+--------+--------+
|Id_1|Id_2|Year|Me_1|Me_2|sum_Me_1|sum_Me_2|
+----+----+----+----+----+--------+--------+
|   A|  XX|2000|   3| 1.0|      21|    17.0|
|   A|  YY|2000|   9| 3.0|      21|    17.0|
|   A|  XX|2001|   4| 9.0|      38|    24.0|
|   A|  YY|2001|   5| 4.0|      38|    24.0|
|   A|  XX|2002|   7| 5.0|      37|    35.0|
|   A|  YY|2002|  10| 2.0|      37|    35.0|
|   A|  XX|2003|   6| 8.0|      28|    22.0|
|   A|  YY|2003|   5| 7.0|      28|    22.0|
+----+----+----+----+----+--------+--------+



## 1.3 min

DS_r := min ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )


In [11]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_by_col)
target_col_name="Me_1"
new_col_name=f"min_{target_col_name}"
df_min=df.withColumn(new_col_name,min(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Id_3|Me_1|Me_2|min_Me_1|
+----+----+----+----+----+--------+
|   A|  YY|2000|   9|   3|       9|
|   A|  YY|2001|   5|   4|       5|
|   A|  YY|2002|  10|   2|       5|
|   A|  YY|2003|   5|   7|       5|
|   A|  XX|2000|   3|   1|       3|
|   A|  XX|2001|   4|   9|       3|
|   A|  XX|2002|   7|   5|       3|
|   A|  XX|2003|   6|   8|       3|
+----+----+----+----+----+--------+



## 1.4 max

DS_r := max ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )

In [12]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_by_col)
target_col_name="Me_1"
new_col_name=f"max_{target_col_name}"
df_min=df.withColumn(new_col_name,max(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Id_3|Me_1|Me_2|max_Me_1|
+----+----+----+----+----+--------+
|   A|  YY|2000|   9|   3|       9|
|   A|  YY|2001|   5|   4|       9|
|   A|  YY|2002|  10|   2|      10|
|   A|  YY|2003|   5|   7|      10|
|   A|  XX|2000|   3|   1|       3|
|   A|  XX|2001|   4|   9|       4|
|   A|  XX|2002|   7|   5|       7|
|   A|  XX|2003|   6|   8|       7|
+----+----+----+----+----+--------+



## 1.5 avg

DS_r := avg ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )


In [13]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name).orderBy(order_by_col)
target_col_name="Me_1"
new_col_name=f"avg_{target_col_name}"
df_min=df.withColumn(new_col_name,avg(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+-----------------+
|Id_1|Id_2|Id_3|Me_1|Me_2|         avg_Me_1|
+----+----+----+----+----+-----------------+
|   A|  YY|2000|   9|   3|              9.0|
|   A|  YY|2001|   5|   4|              7.0|
|   A|  YY|2002|  10|   2|              8.0|
|   A|  YY|2003|   5|   7|             7.25|
|   A|  XX|2000|   3|   1|              3.0|
|   A|  XX|2001|   4|   9|              3.5|
|   A|  XX|2002|   7|   5|4.666666666666667|
|   A|  XX|2003|   6|   8|              5.0|
+----+----+----+----+----+-----------------+



## 1.6 median


DS_r := median ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )

percentile_approx with partitionBy followed by orderBy will do rolling median. To have median of each partition, you must only have partitionBy in your window definition.

In [14]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name)
target_col_name="Me_1"
new_col_name=f"avg_{target_col_name}"
df_min=df.withColumn(new_col_name,percentile_approx(target_col_name,0.5,10000000).over(win_name))
df_min.show()

+----+----+----+----+----+--------+
|Id_1|Id_2|Id_3|Me_1|Me_2|avg_Me_1|
+----+----+----+----+----+--------+
|   A|  YY|2000|   9|   3|       5|
|   A|  YY|2001|   5|   4|       5|
|   A|  YY|2002|  10|   2|       5|
|   A|  YY|2003|   5|   7|       5|
|   A|  XX|2000|   3|   1|       4|
|   A|  XX|2001|   4|   9|       4|
|   A|  XX|2002|   7|   5|       4|
|   A|  XX|2003|   6|   8|       4|
+----+----+----+----+----+--------+



## 1.7 stddev_pop

The operator returns the “population standard deviation” of the input values.

DS_r := stddev_pop ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )

In [15]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name)
target_col_name="Me_1"
new_col_name=f"stddev_pop_{target_col_name}"
df_min=df.withColumn(new_col_name,stddev_pop(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Id_3|Me_1|Me_2|   stddev_pop_Me_1|
+----+----+----+----+----+------------------+
|   A|  YY|2000|   9|   3| 2.277608394786075|
|   A|  YY|2001|   5|   4| 2.277608394786075|
|   A|  YY|2002|  10|   2| 2.277608394786075|
|   A|  YY|2003|   5|   7| 2.277608394786075|
|   A|  XX|2000|   3|   1|1.5811388300841895|
|   A|  XX|2001|   4|   9|1.5811388300841895|
|   A|  XX|2002|   7|   5|1.5811388300841895|
|   A|  XX|2003|   6|   8|1.5811388300841895|
+----+----+----+----+----+------------------+



## 1.8 stddev_samp

The operator returns the “sample standard deviation” of the input values.

DS_r :=  stddev_stamp ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )

In [16]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name)
target_col_name="Me_1"
new_col_name=f"stddev_samp_{target_col_name}"
df_min=df.withColumn(new_col_name,stddev_samp(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Id_3|Me_1|Me_2|  stddev_samp_Me_1|
+----+----+----+----+----+------------------+
|   A|  YY|2000|   9|   3|2.6299556396765835|
|   A|  YY|2001|   5|   4|2.6299556396765835|
|   A|  YY|2002|  10|   2|2.6299556396765835|
|   A|  YY|2003|   5|   7|2.6299556396765835|
|   A|  XX|2000|   3|   1|1.8257418583505536|
|   A|  XX|2001|   4|   9|1.8257418583505536|
|   A|  XX|2002|   7|   5|1.8257418583505536|
|   A|  XX|2003|   6|   8|1.8257418583505536|
+----+----+----+----+----+------------------+



## 1.9 var_pop


DS_r :=  var_pop ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )
The operator returns the “population variance” of the input values

In [17]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name)
target_col_name="Me_1"
new_col_name=f"var_pop_{target_col_name}"
df_min=df.withColumn(new_col_name,var_pop(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Id_3|Me_1|Me_2|      var_pop_Me_1|
+----+----+----+----+----+------------------+
|   A|  YY|2000|   9|   3|            5.1875|
|   A|  YY|2001|   5|   4|            5.1875|
|   A|  YY|2002|  10|   2|            5.1875|
|   A|  YY|2003|   5|   7|            5.1875|
|   A|  XX|2000|   3|   1|2.4999999999999996|
|   A|  XX|2001|   4|   9|2.4999999999999996|
|   A|  XX|2002|   7|   5|2.4999999999999996|
|   A|  XX|2003|   6|   8|2.4999999999999996|
+----+----+----+----+----+------------------+



## 1.10 var_samp


DS_r :=  var_samp ( DS_1 over ( partition by Id_1, Id_2 order by Id_3 ) )

The operator returns the “sample variance” of the input values

In [18]:
partition_col_names=["Id_1","Id_2"]
order_by_col=["Id_3"]
win_name=Window.partitionBy(partition_col_name)
target_col_name="Me_1"
new_col_name=f"var_samp_{target_col_name}"
df_min=df.withColumn(new_col_name,var_samp(target_col_name).over(win_name))
df_min.show()

+----+----+----+----+----+------------------+
|Id_1|Id_2|Id_3|Me_1|Me_2|     var_samp_Me_1|
+----+----+----+----+----+------------------+
|   A|  YY|2000|   9|   3| 6.916666666666667|
|   A|  YY|2001|   5|   4| 6.916666666666667|
|   A|  YY|2002|  10|   2| 6.916666666666667|
|   A|  YY|2003|   5|   7| 6.916666666666667|
|   A|  XX|2000|   3|   1|3.3333333333333326|
|   A|  XX|2001|   4|   9|3.3333333333333326|
|   A|  XX|2002|   7|   5|3.3333333333333326|
|   A|  XX|2003|   6|   8|3.3333333333333326|
+----+----+----+----+----+------------------+

